In [1]:
import os
import subprocess

# Set the working directory to the root of the Git repository
current_dir = os.getcwd()
git_root = subprocess.check_output(["git", "rev-parse", "--show-toplevel"], cwd=current_dir)
git_root = git_root.decode("utf-8").strip()
os.chdir(git_root)
cwd = os.getcwd()

import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import tqdm
from colorama import Fore, Style
import math
from datetime import date
import calendar

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error

import torch
from torch.utils.data import Dataset, DataLoader


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
fpath = os.path.join(cwd, "data", "processed", "data_building_n.parquet")
df = pd.read_parquet(fpath)
df.head(3)

,date_time,tmp,hum,CO2,VOC,room,floor,date,month,hour,day_of_week,is_weekend,season,outside_tmp,outside_hum,outside_rain,outside_snowfall,outside_wind_speed,outside_pressure
0,2023-03-06 00:12:02,20.44,25.06,432,768,002,0,2023-03-06,3,0,0,False,spring,0.489,93.343,0.0,0.0,7.091,1012.0
1,2023-03-06 00:27:08,20.46,24.99,428,758,002,0,2023-03-06,3,0,0,False,spring,0.489,93.343,0.0,0.0,7.091,1012.0
2,2023-03-06 00:42:15,20.44,25.06,429,758,002,0,2023-03-06,3,0,0,False,spring,0.489,93.343,0.0,0.0,7.091,1012.0


### Preprocessing and Feature Engineering (for the NN)

In [3]:
df.columns

Index(['date_time', 'tmp', 'hum', 'CO2', 'VOC', 'room', 'floor', 'date',
       'month', 'hour', 'day_of_week', 'is_weekend', 'season', 'outside_tmp',
       'outside_hum', 'outside_rain', 'outside_snowfall', 'outside_wind_speed',
       'outside_pressure'],
      dtype='object')

#### Time projection
Converting periodic time series data into a format that can be used by a neural network is a bit tricky. The main problem is that the time series is periodic, so the network should be able to understand that the last value is close to the first value.
We therefore project the time series data into a higher dimensional space, where the periodicity is more easily understood. This is done by using the sin and cos functions to encode the time of the year and the time of the day.
See this article for more information: https://towardsdatascience.com/how-to-encode-periodic-time-features-7640d9b21332

In [4]:
def project_date_to_unit_circle(input_date: date):
    year = input_date.year
    passed_days = (input_date - date(year, 1, 1)).days + 1
    nr_of_days_per_year = 366 if calendar.isleap(year) else 365
    position_within_year = passed_days / nr_of_days_per_year
    alpha = position_within_year * math.pi * 2
    year_circle_x = (math.sin(alpha) + 1) / 2
    year_circle_y = (math.cos(alpha) + 1) / 2
    return year_circle_x, year_circle_y

def project_day_of_week_to_unit_circle(input_day_of_week: int):
    alpha = input_day_of_week / 7 * math.pi * 2
    day_of_week_circle_x = (math.sin(alpha) + 1) / 2
    day_of_week_circle_y = (math.cos(alpha) + 1) / 2
    return day_of_week_circle_x, day_of_week_circle_y

# Project the date to a unit circle (year)
df['date_circle_x'], df['date_circle_y'] = zip(*df['date'].apply(project_date_to_unit_circle))

# Project the day_of_week to a unit circle (week)
df['day_of_week_circle_x'], df['day_of_week_circle_y'] = zip(*df['day_of_week'].apply(project_day_of_week_to_unit_circle))

In [5]:
#----- One hot encoding ----------------------------------
# Season
df = pd.get_dummies(df, columns=['season'], prefix='season')

# Floor
df = pd.get_dummies(df, columns=['floor'], prefix='floor')

In [6]:
#----- Resampling ----------------------------------------
df['date_time'] = pd.to_datetime(df['date_time'])
df.set_index('date_time', inplace=True)

df_daily = df.resample('D').mean().dropna()


### Selecting the features

In [7]:
df_daily = df_daily[[
    'tmp', 'hum', 'CO2', 'VOC', 'outside_tmp', 'outside_hum', 'outside_rain',
    'outside_snowfall', 'outside_wind_speed', 'outside_pressure',
    'date_circle_x', 'date_circle_y', 'day_of_week_circle_x',
    'day_of_week_circle_y', 'season_autumn', 'season_spring',
    'season_summer', 'season_winter', 'floor_0', 'floor_1', 'floor_2',
    'floor_3'
    ]]

### Scaling the data

In [8]:
#----- Scaling -------------------------------------------
# Initialize the scalers
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

# Prepare the features and target
features = df_daily.drop(['tmp'], axis=1)
target = df_daily['tmp'].values.reshape(-1, 1)

# Fit and transform the features
scaled_features = scaler_X.fit_transform(features)
scaled_target = scaler_y.fit_transform(target).flatten()

# Create a new DataFrame with scaled features
scaled_df_daily = pd.DataFrame(scaled_features, index=df_daily.index, columns=features.columns)
scaled_df_daily['tmp'] = scaled_target

scaled_df_daily.head()


,hum,CO2,VOC,outside_tmp,outside_hum,outside_rain,outside_snowfall,outside_wind_speed,outside_pressure,date_circle_x,...,day_of_week_circle_y,season_autumn,season_spring,season_summer,season_winter,floor_0,floor_1,floor_2,floor_3,tmp
date_time,,,,,,,,,,,,,,,,,,,,,
2022-05-18,0.469027,0.385416,0.485801,0.750007,0.344018,0.000000,0.0,0.245064,0.633750,0.846859,...,3.568959e-01,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.924290
2022-05-23,0.476494,0.339036,0.297645,0.718637,0.499877,0.011821,0.0,0.299324,0.300974,0.814528,...,1.000000e+00,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.000000
2022-05-24,0.500382,0.120409,0.112113,0.577939,0.417913,0.035806,0.0,0.639309,0.401301,0.807774,...,8.019377e-01,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.878087
2022-05-25,0.309725,0.105410,0.197073,0.612500,0.285711,0.000000,0.0,0.460466,0.549952,0.800928,...,3.568959e-01,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.776042
2022-05-27,0.436637,0.106785,1.000000,0.546858,0.560812,0.008228,0.0,0.704533,0.652704,0.786971,...,6.938894e-18,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.659750


In [9]:
class DailySequenceDataset(Dataset):
    def __init__(self, df_daily, window_size=4):
        """
        Initializes the Dataset.

        :param df_daily: DataFrame with daily resampled data.
        :param window_size: Size of the time window for sequences in days.
        """
        self.df_daily = df_daily
        self.window_size = window_size
        self.X, self.y = self.create_sequences()

    def create_sequences(self):
        """
        Creates sequences of consecutive days.

        :return: Tuple (X, y) where X is the input matrix and y is the output vector.
        """
        X_list = []
        y_list = []

        # Iterate over the DataFrame to find sequences of consecutive days
        for i in range(len(self.df_daily) - self.window_size + 1):
            if all(self.df_daily.index[i + j] == self.df_daily.index[i] + pd.Timedelta(days=j) for j in range(self.window_size)):
                # Extract the input matrix (window_size-1 consecutive days)
                X = self.df_daily.iloc[i:i+self.window_size-1].drop(['tmp'], axis=1).values
                # Extract the output matrix (the last day in the window)
                y = self.df_daily.iloc[i+self.window_size-1]['tmp']
                
                # Append the matrices to the respective lists
                X_list.append(X)
                y_list.append(y)

        # Convert lists to numpy arrays
        X_array = np.array(X_list)
        y_array = np.array(y_list)

        return X_array, y_array

    def __len__(self):
        """
        Returns the number of samples.

        :return: Number of samples.
        """
        return len(self.X)

    def __getitem__(self, idx):
        """
        Returns the sample at the given index.

        :param idx: Index of the sample.
        :return: Tuple (X, y) containing the input data and the target variable.
        """
        X = torch.tensor(self.X[idx], dtype=torch.float32)
        y = torch.tensor(self.y[idx], dtype=torch.float32)
        return X, y

# Initialize the dataset with scaled data
window_size = 4
dataset = DailySequenceDataset(scaled_df_daily, window_size)

# Create a DataLoader
batch_size = 16
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Example of iterating over the DataLoader
for batch in dataloader:
    X_batch, y_batch = batch
    print(X_batch.shape, y_batch.shape)
    break  # Just to demonstrate, remove this in actual usage


torch.Size([16, 3, 21]) torch.Size([16])


### Neural Network Architecture

In [10]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import MinMaxScaler
import torch.nn as nn
import torch.optim as optim

In [11]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out


In [12]:
# Initialize the dataset
window_size = 4
dataset = DailySequenceDataset(scaled_df_daily, window_size)

# Split the dataset into training and testing sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out


In [13]:
# Train the model
# Model parameters
input_size = scaled_features.shape[1]
hidden_size = 50
num_layers = 2
output_size = 1

# Initialize the model, loss function, and optimizer
model = LSTMModel(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_dataloader:
        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch.unsqueeze(1))
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


In [ ]:
# Evaluate the model on the test set
model.eval()
test_predictions = []
test_actuals = []
with torch.no_grad():
    for X_batch, y_batch in test_dataloader:
        outputs = model(X_batch)
        test_predictions.append(outputs.squeeze().cpu().numpy())
        test_actuals.append(y_batch.cpu().numpy())

# Flatten the lists
test_predictions = np.concatenate(test_predictions)
test_actuals = np.concatenate(test_actuals)

# Inverse transform the predictions and actuals to original scale
test_predictions = scaler_y.inverse_transform(test_predictions.reshape(-1, 1)).flatten()
test_actuals = scaler_y.inverse_transform(test_actuals.reshape(-1, 1)).flatten()

# Calculate and print the MSE and RMSE for the test data
test_mse = mean_squared_error(test_actuals, test_predictions)
test_rmse = np.sqrt(test_mse)
print(f'Test MSE: {test_mse:.4f}, Test RMSE: {test_rmse:.4f}')


Test MSE: 0.1493, Test RMSE: 0.3864


In [ ]:
test_actuals, test_predictions

(array([20.910542, 20.555653, 21.304731, 20.472727, 20.41582 , 21.202549,
        20.654362, 21.550238, 21.446922, 20.366251, 20.492321, 20.703049,
        20.573921], dtype=float32),
 array([21.046564, 21.025482, 21.45443 , 20.78753 , 21.237274, 21.215248,
        20.964836, 21.18085 , 20.966024, 20.783794, 20.681116, 21.174911,
        20.664803], dtype=float32))

In [ ]:
# Evaluate the model on the test set



model.eval()
predictions = []
actuals = []
with torch.no_grad():
    for X_batch, y_batch in test_dataloader:
        outputs = model(X_batch)
        predictions.append(outputs.squeeze().cpu().numpy())
        actuals.append(y_batch.cpu().numpy())

# Flatten the lists
predictions = np.concatenate(predictions)
actuals = np.concatenate(actuals)

# Inverse transform the predictions and actuals to original scale
predictions = scaler_y.inverse_transform(predictions.reshape(-1, 1)).flatten()
actuals = scaler_y.inverse_transform(actuals.reshape(-1, 1)).flatten()

# Calculate and print the MSE and RMSE
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
print(f'MSE: {mse:.4f}, RMSE: {rmse:.4f}')


MSE: 0.0700, RMSE: 0.2646


In [ ]:
actuals, predictions

(array([20.910542, 20.555653, 21.304731, 20.472727, 20.41582 , 21.202549,
        20.654362, 21.550238, 21.446922, 20.366251, 20.492321, 20.703049,
        20.573921], dtype=float32),
 array([20.938267, 20.684793, 21.226948, 20.544085, 20.820656, 21.13143 ,
        20.704638, 21.144112, 20.732079, 20.51746 , 20.511965, 20.787432,
        20.510687], dtype=float32))